<a href="https://colab.research.google.com/github/nyayuchika/pcd-coursework/blob/master/PCD_Assignment01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upsampling

In [1]:
#Generate Matrix
import numpy as np
m = 0
n = 0
m = input("Masukkan banyak baris = ")
n = input("Masukkan banyak kolom = ")
print(f"Banyak baris {m}")
print(f"Banyak kolom {n}")
m = int(m)
n = int(n)
a = np.random.randint(10, size=(m,n))
print(a)

Masukkan banyak baris = 3
Masukkan banyak kolom = 3
Banyak baris 3
Banyak kolom 3
[[7 0 4]
 [4 3 1]
 [4 9 9]]


##Nearest Neighbor

In [2]:
print(a)

[[7 0 4]
 [4 3 1]
 [4 9 9]]


In [3]:
k = 2



##Bilinear

##Bicubic

#Downsampling

In [4]:
#Generate Matrix
import numpy as np
m = 0
n = 0
m = input("Masukkan banyak baris = ")
n = input("Masukkan banyak kolom = ")
print(f"Banyak baris {m}")
print(f"Banyak kolom {n}")
m = int(m)
n = int(n)
b = np.random.randint(10, size=(m,n))
print(b)

Masukkan banyak baris = 4
Masukkan banyak kolom = 4
Banyak baris 4
Banyak kolom 4
[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]


##Max

In [85]:
print(b)
print(b.shape)

[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]
(4, 4)


In [86]:
# k = 2

b_m1 = b[0:2,0:2]
print(b_m1)
max_b1 = np.max(b_m1)
print(max_b1)

[[0 3]
 [5 0]]
5


In [91]:
b_m2 = b[0:2,2:4]
print(b_m2)
max_b2 = np.max(b_m2)
print(max_b2)

[[0 2]
 [6 5]]
6


In [92]:
b_m3 = b[2:5,0:2]
print(b_m3)
max_b3 = np.max(b_m3)
print(max_b3)

[[5 7]
 [5 8]]
8


In [93]:
b_m4 = b[2:5,2:4]
print(b_m4)
max_b4 = np.max(b_m4)
print(max_b4)

[[9 4]
 [9 0]]
9


In [94]:
b_max = np.array([[max_b1,max_b2],[max_b3,max_b4]])
print(f"Matriks Downsampling Max: \n",b_max)

Matriks Downsampling Max: 
 [[5 6]
 [8 9]]


##Median

In [71]:
import statistics
print(b)

[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]


In [76]:
b_med1 = b[0:2,2:4]
print(b_med1)
b_med1 = b_med1.flatten()
b_med1 = np.sort(b_med1)
print(b_med1)
med_b1 = int(statistics.median(b_med1))
print(med_b1)

[[0 2]
 [6 5]]
[0 2 5 6]
3


In [80]:
b_med2 = b[0:2,2:5]
print(b_med2)
b_med2 = b_med2.flatten()
b_med2 = np.sort(b_med2)
print(b_med2)
med_b2 = int(statistics.median(b_med2))
print(med_b2)

[[0 2]
 [6 5]]
[0 2 5 6]
3


In [78]:
b_med3 = b[2:5,0:2]
print(b_med3)
b_med3 = b_med3.flatten()
b_med3 = np.sort(b_med3)
print(b_med3)
med_b3 = int(statistics.median(b_med3))
print(med_b3)

[[5 7]
 [5 8]]
[5 5 7 8]
6


In [81]:
b_med4 = b[2:5,2:5]
print(b_med4)
b_med4 = b_med4.flatten()
b_med4 = np.sort(b_med4)
print(b_med4)
med_b4 = int(statistics.median(b_med4))
print(med_b4)

[[9 4]
 [9 0]]
[0 4 9 9]
6


In [83]:
b_median = np.array([[med_b1,med_b2],[med_b3,med_b4]])
print(f"Matriks Downsampling Median: \n",b_median)

Matriks Downsampling Median: 
 [[3 3]
 [6 6]]


##Mean

In [54]:
print(b)
print(b.shape)

[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]
(4, 4)


In [55]:
# k = 2

b_avg1 = b[0:2,0:2]
print(b_avg1)
b_avg1 = b_avg1.flatten()
mean_b1 = int(sum(b_avg1)/len(b_avg1))
print(mean_b1)

[[0 3]
 [5 0]]
2


In [56]:
b_avg2 = b[0:2,2:4]
print(b_avg2)
b_avg2 = b_avg2.flatten()
mean_b2 = int(sum(b_avg2)/len(b_avg2))
print(mean_b2)

[[0 2]
 [6 5]]
3


In [57]:
b_avg3 = b[2:5,0:2]
print(b_avg3)
b_avg3 = b_avg3.flatten()
mean_b3 = int(sum(b_avg3)/len(b_avg3))
print(mean_b3)

[[5 7]
 [5 8]]
6


In [58]:
b_avg4 = b[2:5,2:4]
print(b_avg4)
b_avg4 = b_avg4.flatten()
mean_b4 = int(sum(b_avg4)/len(b_avg4))
print(mean_b4)

[[9 4]
 [9 0]]
5


In [59]:
b_mean = np.array([[mean_b1,mean_b2],[mean_b3,mean_b4]])
print(f"Matriks Downsampling Mean \n",b_mean)

Matriks Downsampling Mean 
 [[2 3]
 [6 5]]


##Conclusion

In [95]:
print(f"""The result from downsampling method:
Before downsampling: \n{b} \n\nAfter downsampling:
1. Max: \n{b_max} \n\n2. Median: \n{b_median} \n\n3. Mean: \n{b_mean}""")

The result from downsampling method:
Before downsampling: 
[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]] 

After downsampling:
1. Max: 
[[5 6]
 [8 9]] 

2. Median: 
[[3 3]
 [6 6]] 

3. Mean: 
[[2 3]
 [6 5]]
